In [1]:
#conda install geopandas

In [2]:
import json
import geopandas as gpd
import pandas as pd
import numpy as np

# Playground and Green Spaces Dataset Wrangling

In [3]:
pgs_fname = "./Data/airesJeux.json"
pgs_gdf = gpd.read_file(pgs_fname)
pgs_gdf.head()

,gml_id,objectid,id_ajeu,annee,borne_eau,eclairage,nom,nom_voie,notes,numero,sev,echelle,espace_publique,numsite,nuquart,nusect,geometry
0,gev_ajeu.1,1,108,0.0,,,CRECHE PAULINE KERGOMARD,de Gascogne,,0186AJEUX01,Equipe 18,500,NON,0186,10,18,"MULTIPOLYGON (((-1.71742 48.12282, -1.71742 48..."
1,gev_ajeu.2,2,163,0.0,,,JARDIN DU BONHEUR,du Gros chêne,,0259AJEUX01,Equipe 11,500,OUI,0259,5,11,"MULTIPOLYGON (((-1.65895 48.12563, -1.65904 48..."
2,gev_ajeu.3,3,83,0.0,,,PROMENADE DES DOMBES,,,0362AJEUX02,Equipe 13,500,OUI,0362,10,13,"MULTIPOLYGON (((-1.70714 48.12337, -1.70713 48..."
3,gev_ajeu.4,4,105,0.0,,,SQUARE DE LA MARCHE,de Gascogne,,0184AJEUX01,Equipe 18,500,OUI,0184,10,18,"MULTIPOLYGON (((-1.71404 48.12236, -1.71399 48..."
4,gev_ajeu.5,5,19,0.0,,,ALLEE DE BRNO,de Brno,,0254AJEUX04,Equipe 11,500,OUI,0254,5,11,"MULTIPOLYGON (((-1.66103 48.12754, -1.66110 48..."


In [4]:
pgs_gdf.rename(columns={'objectid':'Object_ID', 'borne_eau':'Water',
                       'eclairage':'Lighting', 'nom':'Object_Name',
                       'espace_publique':'Public_Space',
                       'nusect':'Neighborhood_ID'}, inplace=True)
pgs_gdf.drop(['gml_id', 'id_ajeu', 'annee', 'nom_voie', 'notes',
             'numero', 'sev', 'echelle', 'numsite', 'nuquart'], 
             axis=1, inplace=True)
pgs_gdf.head()

,Object_ID,Water,Lighting,Object_Name,Public_Space,Neighborhood_ID,geometry
0,1,,,CRECHE PAULINE KERGOMARD,NON,18,"MULTIPOLYGON (((-1.71742 48.12282, -1.71742 48..."
1,2,,,JARDIN DU BONHEUR,OUI,11,"MULTIPOLYGON (((-1.65895 48.12563, -1.65904 48..."
2,3,,,PROMENADE DES DOMBES,OUI,13,"MULTIPOLYGON (((-1.70714 48.12337, -1.70713 48..."
3,4,,,SQUARE DE LA MARCHE,OUI,18,"MULTIPOLYGON (((-1.71404 48.12236, -1.71399 48..."
4,5,,,ALLEE DE BRNO,OUI,11,"MULTIPOLYGON (((-1.66103 48.12754, -1.66110 48..."


In [5]:
pgs_gdf.loc[(pgs_gdf['Water'] == 'Oui'), 'Water'] = 'YES'
pgs_gdf.loc[(pgs_gdf['Water'] == 'Non'), 'Water'] = 'NO'

pgs_gdf.loc[(pgs_gdf['Lighting'] == 'Oui'), 'Lighting'] = 'YES'
pgs_gdf.loc[(pgs_gdf['Lighting'] == 'OUI'), 'Lighting'] = 'YES'
pgs_gdf.loc[(pgs_gdf['Lighting'] == 'Non'), 'Lighting'] = 'NO'

In [6]:
# Limit the analysis to public spaces
pgs_gdf = pgs_gdf.loc[(pgs_gdf['Public_Space'] != 'NON')]
pgs_gdf.dropna(subset=['Public_Space'], axis=0, inplace=True)
pgs_gdf.reset_index(drop=True, inplace=True)
pgs_gdf

,Object_ID,Water,Lighting,Object_Name,Public_Space,Neighborhood_ID,geometry
0,2,,,JARDIN DU BONHEUR,OUI,11,"MULTIPOLYGON (((-1.65895 48.12563, -1.65904 48..."
1,3,,,PROMENADE DES DOMBES,OUI,13,"MULTIPOLYGON (((-1.70714 48.12337, -1.70713 48..."
2,4,,,SQUARE DE LA MARCHE,OUI,18,"MULTIPOLYGON (((-1.71404 48.12236, -1.71399 48..."
3,5,,,ALLEE DE BRNO,OUI,11,"MULTIPOLYGON (((-1.66103 48.12754, -1.66110 48..."
4,6,,,JARDIN ARC EN CIEL,OUI,11,"MULTIPOLYGON (((-1.65856 48.12700, -1.65860 48..."
...,...,...,...,...,...,...,...
220,2800,None,None,JARDINS DES ORMEAUX,OUI,19,"MULTIPOLYGON (((-1.66834 48.10008, -1.66824 48..."
221,2803,None,None,JARDIN SOPHIE GERMAIN,OUI,15,"MULTIPOLYGON (((-1.65341 48.10904, -1.65331 48..."
222,2824,None,None,SQUARE DU PRE PERCHE,OUI,10,"MULTIPOLYGON (((-1.68159 48.10664, -1.68154 48..."
223,2828,None,None,RUE ADOLPHE LERAY,OUI,19,"MULTIPOLYGON (((-1.66095 48.10006, -1.66087 48..."


In [7]:
import folium
from geopy import geocoders
from geopy.geocoders import GoogleV3

In [8]:
# Getting coordinates of the City of Rennes
with open("google_api.txt") as file:
    api_key = file.readline()
    
g = GoogleV3(api_key=api_key)

In [9]:
location = g.geocode('Rennes, France')
latitude = location.latitude
longitude = location.longitude
print(f"{latitude},{longitude}")

48.117266,-1.6777926


In [10]:
pgs_map = folium.Map(location=[latitude, longitude], 
                     zoom_start=12,
                     control_scale=True)

folium.GeoJson(pgs_gdf, name='geojson').add_to(pgs_map)

for geo in pgs_gdf['geometry']:
    lat = geo.centroid.y
    lng = geo.centroid.x
    label=''
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(pgs_map)

folium.LayerControl().add_to(pgs_map)

pgs_map

In [11]:
pgs_gdf['Latitude'] = pgs_gdf['geometry'].centroid.y
pgs_gdf['Longitude'] = pgs_gdf['geometry'].centroid.x

In [12]:
pgs_gdf.head()

,Object_ID,Water,Lighting,Object_Name,Public_Space,Neighborhood_ID,geometry,Latitude,Longitude
0,2,,,JARDIN DU BONHEUR,OUI,11,"MULTIPOLYGON (((-1.65895 48.12563, -1.65904 48...",48.125615,-1.659514
1,3,,,PROMENADE DES DOMBES,OUI,13,"MULTIPOLYGON (((-1.70714 48.12337, -1.70713 48...",48.123342,-1.707076
2,4,,,SQUARE DE LA MARCHE,OUI,18,"MULTIPOLYGON (((-1.71404 48.12236, -1.71399 48...",48.122309,-1.714101
3,5,,,ALLEE DE BRNO,OUI,11,"MULTIPOLYGON (((-1.66103 48.12754, -1.66110 48...",48.127573,-1.661070
4,6,,,JARDIN ARC EN CIEL,OUI,11,"MULTIPOLYGON (((-1.65856 48.12700, -1.65860 48...",48.127121,-1.658643


In [13]:
pgs_gdf.to_file("./Data/cleaned_data/playground_and_greenspaces.geojson", driver='GeoJSON')